In [3]:
import geopandas
import geodatasets

In [4]:
# nyc_trees = geopandas.read_file("data/2015 Street Tree Census - Tree Data.geojson")
# note: you need to install pyarrow for this to work
# nyc_trees = geopandas.read_parquet("data/nyc_trees.parquet")

In [6]:
import sys 
!{sys.executable} -m pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 10.5 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
# to_parquet is not supported in production yet.
# using parquet reduces the read of the file to < 1 min.
# without using parquet, read_file() takes ~6 min
nyc_trees.to_parquet('data/nyc_trees.parquet')

In [ ]:
# TODO: Split up nyc_trees into bk_trees, mh_trees, queens_trees, si_trees, bronx_trees

In [ ]:
nyc_trees.head()

In [ ]:
nyc_trees.columns

In [ ]:
nyc_trees['borocode'].head()

In [ ]:
nyc_trees['borocode'] = nyc_trees['borocode'].astype("int64")

In [ ]:
boros = geopandas.read_file(geodatasets.get_path("nybb"))
boros.head()

In [ ]:
# TODO join boros and nyc_trees on boros,BoroCode and nyc_trees,borocode

In [ ]:
boros.crs

In [ ]:
nyc_trees.crs

In [ ]:
# going to change the crs of boros since it is smaller
boros = boros.to_crs(nyc_trees.crs)

In [ ]:
boros.crs

In [ ]:
boro_names = boros[['BoroCode', 'BoroName']]

In [ ]:
boro_names = boro_names.rename(columns={'BoroCode': 'borocode'})

In [ ]:
# can we use `join()` instead of merge?
nyc_trees = nyc_trees.merge(boro_names, on='borocode')

In [ ]:
nyc_trees['BoroName'].head()

### Which Borough Has the Most Trees?

In [ ]:
tree_count = nyc_trees.groupby(['BoroName']).sum()

In [ ]:
tree_count = tree_count.rename(columns={'borocode': 'count'})

In [ ]:
tree_count.sort_values(by=['count'])

In [ ]:
# get the geometries back onto tree_count
tree_count = boros.merge(tree_count, on='BoroName')

In [ ]:
# TODO: display the above as a chloropleth map
base = boros.plot(color='white', edgecolor='blue')

In [ ]:
tree_count.plot(column="count",  legend=True)

## What is the area of each borough?

And which borough has the highest *density* of trees?

In [ ]:
# we need to convert Geometry column back to a geographic CRS to be able to get areas
boros_geographic = boros.to_crs("EPSG:2263")

What units will the area be in?
- you can find out in this case from the `.name` of the `.crs` object
- in this case the name tells us the units are feet (ft)
- when we take the area, we will get feet squared (ft^2)

In [ ]:
boros_geographic.crs.name

In [ ]:
boros_geographic.geometry.area

In [ ]:
# add tree count to boros_geographic
# create a new gdf with BoroName, area, count, 
# how to merge in the tree_count['count'] based on BoroName?
# boros_geographic['tree_count'] = tree_count['count']